In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=e85d510154b7c3a44b8eb1aae2f33d5df5dbb73393f50c9c6d74e64166c04973
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.

✅   Downloaded https://dl.fbaipublicfiles.com/laser/mod

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

93langs.fcodes  93langs.fvocab  bilstm.93langs.2018-12-26.pt


In [3]:
corpus_emb_quran = np.load('../input/assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('../input/assets/en_emb_bukhari_muslim.npy')

In [4]:
en_bn_quran_tafsir = pd.read_csv('../input/assets/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('../input/assets/en_bn_translated_sahih_hadiths_filtered.csv')

In [5]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [6]:
en_bn_bukhari_muslim.head(1)

,source,chapter_no,hadith_no,chapter,text_ar,text_en,narrators,text_bn
0,Sahih Bukhari,1,1,Revelation,حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...


In [7]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_hadith, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore
        
    return find

In [8]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...


In [9]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=size,language = 'en',metric = 'dot',query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_quran, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_bn_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_bn_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_bn_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = str(en_bn_quran_tafsir['Verse'][index]) 
        find.loc[i,'Tafseer'] = str(en_bn_quran_tafsir['Tafseer'][index]) 
        find.loc[i,'ar_text'] = str(en_bn_quran_tafsir['ar_text'][index])
        
        find.loc[i,'আল_বায়ান'] = str(en_bn_quran_tafsir['আল_বায়ান'][index])
        find.loc[i,'tafsir_bayan'] = str(en_bn_quran_tafsir['tafsir_bayan'][index])
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore

    return find,mlt_hadiths

In [10]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("এই হচ্ছে মারইয়াম পুত্র ঈসা। এটাই সঠিক বক্তব্য",size=2,language = 'en')

CPU times: user 658 ms, sys: 151 ms, total: 809 ms
Wall time: 533 ms


In [11]:
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,Mary,19,34,"Such was Jesus, son of Mary: (this is) a state...",That is Jesus son of Mary a statement of truth...,ذٰلِکَ عِیۡسَی ابۡنُ مَرۡیَمَ ۚ قَوۡلَ الۡحَق...,"এই হচ্ছে মারইয়াম পুত্র ঈসা। এটাই সঠিক বক্তব্য,...",(৩৪) এই হল মারয়্যাম তনয় ঈসা (এর বৃত্তান্ত)। (আ...,0.012600
1,The Believers,23,50,And We made the son of Mary and his mother a p...,And We made the son of Mary Jesus and his moth...,وَ جَعَلۡنَا ابۡنَ مَرۡیَمَ وَ اُمَّهٗۤ اٰیَۃ...,আর আমি মারইয়াম-পুত্র ও তার মাকে নিদর্শন বানালা...,(৫০) এবং আমি মারয়্যাম তনয় (ঈসা) ও তার জননীকে ক...,0.010997


In [12]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,64,4193,Military Expeditions led by the Prophet pbuh A...,حدثنا عبد العزيز بن عبد الله، حدثنا إبراهيم بن...,Narrated `Aisha: Whenever...,"` আয়েশা (রাঃ) বলেন, যখনই আল্লাহর রসূল (সাঃ) ক...","'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.004165
1,Sahih Bukhari,54,2732,Conditions,حدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخبر...,Narrated Al-Miswar bin Makhrama and Marwan: ...,আল-মিসওয়ার বিন মখরমা ও মারওয়ান বর্ণনা করেছেন...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.003988


In [13]:
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("আমি কি পুরস্কার পেতে পারি?",language = 'bn')
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Event,56,24,Reward for what they used to do.,a reward jazā’an is an object denoting reason ...,جَزَآءًۢ بِمَا کَانُوۡا یَعۡمَلُوۡنَ ﴿۲۴﴾\nج...,তারা যে আমল করত তার প্রতিদানস্বরূপ।,(২৪) তাদের কর্মের পুরস্কার স্বরূপ।-তাফসীরে আহস...,0.009955
1,The Reality,69,47,And not one of you could have held Us off from...,and not one of you ahadin is the subject of mā...,فَمَا مِنۡکُمۡ مِّنۡ اَحَدٍ عَنۡهُ حٰجِزِیۡنَ...,অতঃপর তোমাদের মধ্যে কেউই তাকে রক্ষা করার থাকত ...,(৪৭) অতঃপর তোমাদের মধ্যে এমন কেউ নেই যে তাকে র...,0.009949
2,The Cave,18,102,Do the disbelievers reckon that they can choos...,Do the disbelievers reckon that they can take ...,اَفَحَسِبَ الَّذِیۡنَ کَفَرُوۡۤا اَنۡ یَّتَّخ...,"যারা কুফরী করছে, তারা কি মনে করেছে যে, তারা আম...",(১০২) যারা সত্য প্রত্যাখ্যান করেছে তারা কি মনে...,0.009504
3,The Prophets,21,111,And I know not but that this may be a trial fo...,I do not know; perhaps that which I have notif...,وَ اِنۡ اَدۡرِیۡ لَعَلَّهٗ فِتۡنَۃٌ لَّکُمۡ و...,আর আমি জানি না হয়তো তা তোমাদের জন্য এক পরীক্ষা...,"(১১১) আমি জানি না, হয়তো এটা তোমাদের জন্য এক পর...",0.009492
4,The Prophets,21,21,Or have they chosen gods from the earth who ra...,Or am functions with the meaning of bal ‘nay’ ...,اَمِ اتَّخَذُوۡۤا اٰلِهَۃً مِّنَ الۡاَرۡضِ هُ...,"তারা যেসব মাটির দেবতা গ্রহণ করেছে, সেগুলি কি ম...",(২১) ওরা মাটি হতে তৈরী যে সব উপাস্য গ্রহণ করেছ...,0.009294
5,He Frowned,80,4,Or take heed and so the reminder might avail h...,or be admonished yadhdhakkar the original tā’ ...,اَوۡ یَذَّکَّرُ فَتَنۡفَعَهُ الذِّکۡرٰی ؕ﴿۴﴾\...,"অথবা উপদেশ গ্রহণ করত, ফলে সে উপদেশ তার উপকারে ...","৪। অথবা উপদেশ গ্রহণ করত, ফলে তা তার উপকারে আসত...",0.009285
6,He Frowned,80,13,On honoured leaves,on leaves fī suhufin is the second predicate o...,فِیۡ صُحُفٍ مُّکَرَّمَۃٍ ﴿ۙ۱۳﴾\nفی صحف مکرمۃ ...,এটা আছে সম্মানিত সহীফাসমূহে।*,১৩। সম্মানিত পত্রসমূহে (লওহে মাহফূযে তা লিপিবদ...,0.009268
7,Qâf,50,29,The sentence that cometh from Me cannot be cha...,The word that comes from Me in this respect ca...,مَا یُبَدَّلُ الۡقَوۡلُ لَدَیَّ وَ مَاۤ اَنَا...,"‘আমার কাছে কথা রদবদল হয় না, আর আমি বান্দার প্র...",(২৯) আমার কথার রদ-বদল হয় না[1] এবং আমি আমার বা...,0.009257
8,The Pen,68,9,"Who would have had thee compromise, that they ...",They desire they yearn that law relates to the...,وَدُّوۡا لَوۡ تُدۡهِنُ فَیُدۡهِنُوۡنَ ﴿۹﴾\nود...,"তারা কামনা করে, যদি তুমি আপোষকামী হও, তবে তারা...","(৯) তারা চায় যে, তুমি নমনীয় হও। তাহলে তারাও নম...",0.009231
9,The Poets,26,14,"And they have a crime against me, so I fear th...",And I have sinned against them by my slaying o...,وَ لَهُمۡ عَلَیَّ ذَنۡۢبٌ فَاَخَافُ اَنۡ یَّق...,‘আর আমার বিরুদ্ধে তাদের কাছে একটি অপরাধের অভিয...,"(১৪) আমার বিরুদ্ধে ওদের এক অভিযোগ আছে, আমি আশং...",0.009218


In [14]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,87,6918,Punishment of Disbelievers at War with Allah a...,حدثنا عبد العزيز بن عبد الله، حدثني إبراهيم بن...,Narrated Ibn `Abbas: I us...,ইবনে আব্বাস বলেনঃ আমি মুহাজিরদের একদলকে শিক্ষা...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.007022
1,Sahih Muslim,51,7196,The Book Pertaining to Repentance and Exhortat...,حدثنا حبان بن موسى، أخبرنا عبد الله بن المبارك...,"Sa'id b. Musayyib, 'Urwa b. Zubair, 'Alqama b...","সাঈদ খ. মুসাইব, 'উরুওয়া খ. জুবাইর, 'আলকামা খ....","Hiban bin Musa bin Sawar ,'Abdullah bin Mubara...",0.006431
2,Sahih Bukhari,52,2661,Witnesses,حدثنا أبو الربيع، سليمان بن داود وأفهمني بعضه ...,Narrated Aisha: (the wife...,আয়েশা (আঃ) এর বৎসরী বর্ণনা: (নবী (সাঃ) এর স্ত...,"Sulaiman bin Da'ud al 'Atki al Zahrani ,Falayh...",0.006297
3,Sahih Bukhari,64,4193,Military Expeditions led by the Prophet pbuh A...,حدثنا عبد العزيز بن عبد الله، حدثنا إبراهيم بن...,Narrated `Aisha: Whenever...,"` আয়েশা (রাঃ) বলেন, যখনই আল্লাহর রসূল (সাঃ) ক...","'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.006281
4,Sahih Bukhari,54,2732,Conditions,حدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخبر...,Narrated Al-Miswar bin Makhrama and Marwan: ...,আল-মিসওয়ার বিন মখরমা ও মারওয়ান বর্ণনা করেছেন...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.004997
5,Sahih Muslim,33,4779,The Book of Jihad and Expedition,حدثنا أبو بكر بن أبي شيبة، حدثنا هاشم بن القاس...,It has been narrated on the authority of Ibn ...,এটি ইবনে সালামার কর্তৃত্বে বর্ণিত হয়েছে। তিনি...,"Abu Bakr bin Abi Shayba ,Hashim bin al Qasim b...",0.004944
6,Sahih Bukhari,60,3342,Prophets,قال عبدان أخبرنا عبد الله، أخبرنا يونس، عن الز...,Narrated Anas (ra): Abu ...,"আনাস (রাঃ) এর কাহিনীঃ আবু দাহর (রাঃ) বলেন, রাস...","'Abdullah bin 'Uthman ''Aabadan ,'Abdullah bin...",0.004739
7,Sahih Bukhari,60,3364,Prophets,وحدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخب...,Narrated Ibn `Abbas: The ...,ইবনে আব্বাস বর্ণনা করেছেন: প্রথম মহিলা যিনি বে...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.004442
8,Sahih Bukhari,65,4797,Prophetic Commentary on the Qur'an Tafseer of ...,حدثنا يحيى بن بكير، حدثنا الليث، عن يونس، عن ا...,Narrated Aisha: (The wife...,"আয়েশা (আঃ) বলেন, 'নিসন্দেহে রাসূল (সাঃ) যখনই ...","Yahya bin Yahya bin Bukayr ,al Laith bin Sa'd ...",0.003621
9,Sahih Bukhari,65,4595,Prophetic Commentary on the Qur'an Tafseer of ...,حدثني إبراهيم بن موسى، عن هشام، عن معمر،‏.‏ وح...,Narrated Ibn `Abbas: Abu ...,ইবনে আব্বাস বর্ণনা করেছেন: আবু সুফিয়ান আমাকে ...,"Ibrahim bin Musa al Razi al Fara' ,Hisham bin ...",0.003519


In [15]:
import site
loc = site.getsitepackages() 
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'